In [9]:
import pandas as pd
import numpy as np

In [10]:
# import dataset into a pandas data frame

df = pd.read_csv('../41467_2022_32829_MOESM5_ESM.csv')
df.head()

,ID,DNA1 [counts],DNA2 [counts],DNA3 [counts],RNA1 [counts],RNA2 [counts],RNA3 [counts],TX1 [au],TX2 [au],TX3 [au],...,high quality,Observed log(TX/Txref),Predicted log(TX/Txref),dG10,dG35,dGDisc,dGITR,dGEXT10,dGSPAC,dGUP
0,0,8263,7261,5173,16341,10320,13506,2.258071,1.523795,1.545541,...,Yes,-3.386326,-3.844827,-1.781524,-1.477218,-0.106428,-0.021112,0.191352,-0.0924,0.400862
1,1,5600,4886,3264,10986,7250,10800,2.240001,1.590845,1.958709,...,Yes,-3.503140,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
2,2,7898,6790,4752,19572,32204,30585,2.829533,5.084911,3.810029,...,Yes,-4.207206,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
3,3,10651,9875,6466,15734,16246,18908,1.686729,1.763814,1.731036,...,Yes,-3.392439,-3.877808,-1.781524,-1.477218,-0.139409,-0.021112,0.191352,-0.0924,0.400862
4,4,12188,10793,6965,28609,21796,26803,2.680198,2.165100,2.278025,...,Yes,-3.698903,-3.672384,-1.781524,-1.477218,0.066015,-0.021112,0.191352,-0.0924,0.400862


In [11]:
# All input and output data

X = df[['UP', 'h35', 'spacs', 'h10', 'disc', 'ITR']]
y = df['Observed log(TX/Txref)']

X.head()

,UP,h35,spacs,h10,disc,ITR
0,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CCCCGCGG,CTCTACCTTAGTTTGTACGTT
1,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGCGGCGG,CTCTACCTTAGTTTGTACGTT
2,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGCGCCCG,CTCTACCTTAGTTTGTACGTT
3,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,GCGGCGGC,CTCTACCTTAGTTTGTACGTT
4,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGGGGGGC,CTCTACCTTAGTTTGTACGTT


In [12]:
# remove all rows with spacer sequences that are not 16-18 nucleotides long


_df = df[(df['spacs'].str.len() >= 15) & (df['spacs'].str.len() <= 19)]


X = _df[['UP', 'h35', 'spacs', 'h10', 'disc', 'ITR']]
y = _df['Observed log(TX/Txref)']

print(f'Removed {df.shape[0] - _df.shape[0]} rows')


Removed 140 rows


In [13]:
# Function to one-hot encode DNA sequences, including padding 0's

def padded_one_hot_encode(sequence):
    mapping = {'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1], '0': [0,0,0,0]}
    encoding = []
    for nucleotide in sequence:
         encoding += [mapping[nucleotide]]
    return encoding

In [14]:
upstream_padding = {}

for col in X.columns:
    max_len = X[col].apply(len).max()
    upstream_padding[col] = np.array([padded_one_hot_encode('0' * (max_len - len(seq)) + seq) for seq in X[col]])

# Concatenate the one-hot encoded, upstream-padded sequences
X = np.concatenate([upstream_padding[col] for col in X.columns], axis=1)

In [15]:
X[0]

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0,

In [16]:
# split the data in training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

ModuleNotFoundError: No module named 'sklearn'

In [10]:
# Import necessary libraries
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import Dropout

ModuleNotFoundError: No module named 'keras'

In [82]:
def initModel(hidden_layers, neurons, epochs=150, batch_size=32):
    # Define RNN model architecture
    regressor = Sequential()
    regressor.add(Input(shape=X.shape[1:])) # dynamically generated input shape based on X data
    for i in range(hidden_layers-1):
        regressor.add(LSTM(neurons, return_sequences=True))
    regressor.add(LSTM(neurons))
    regressor.add(Dense(1, activation='linear'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    regressor.compile(optimizer=optimizer, loss='mean_squared_error')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history = regressor.fit(X_train,
                            y_train,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping])

    # Evaluate the model
    return regressor, regressor.evaluate(X_test, y_test)

In [14]:
hidden_layers = 1
neurons = 80
epochs = 150
batch_size = 64

print(f"Hidden Layers: {hidden_layers}, Neurons {neurons}")
regressor, mse = initModel(hidden_layers, neurons, epochs, batch_size)

print(f"MSE: {mse}")
regressor.save("Hyperparameter_tuned.keras")


Hidden Layers: 1, Neurons 80
Epoch 1/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.5906 - val_loss: 0.5373
Epoch 2/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.5154 - val_loss: 0.4297
Epoch 3/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.4533 - val_loss: 0.3338
Epoch 4/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.3222 - val_loss: 0.3337
Epoch 5/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.3222 - val_loss: 0.2650
Epoch 6/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.2527 - val_loss: 0.2304
Epoch 7/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.2437 - val_loss: 0.2638
Epoch 8/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 0.2473 - val_loss: 0.2350
Epoch 9/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.2525 - val_loss: 0.2362
Epoch 10/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 12s 70ms/step - loss: 0.2397 - val_loss: 0.2458
Epoch 11/150
167/167 ━━━━━━━━━━━━━━━━━━━━ 24s 89ms/step - loss: 0.2308 - val_loss: 0.2165